<a href="https://colab.research.google.com/github/SotiriosZiakas/Churn-Modelling-ANN-/blob/main/Artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [4]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Indicate the index of the column that will be encoded
X[:,2] = le.fit_transform(X[:,2])


In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 0)

### Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
"""
units: the number of neurons
activation: the activation function, 'relu' is the rectifier function
"""
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the output layer

In [14]:
"""
1 unit (neuron) can encode the dependent variable which is binary
we use the 'sigmoid' function which guarantees that the output of this unit will always be between 0 and 1
"""
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [15]:
"""
optimizer: updates the weights (best are the ones that perform stochastic gradient descent)

loss function: computes the difference between the predictions and the real results
for binary classification problems we always use 'binary_crossentropy' 
whereas for non-binary classification we use 'categorical_crossentropy

metrics: evaluation metrics - a list of metrics that will evaluate the ann during the training

"""
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
""" 
batch_size: instead of comparing the predictions with the real results one by one, 
we do that with several predictions at a time
epochs: the number of times the entire dataset is passed forward and backward through the neural network

"""
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Use our ANN model to predict if the customer with the following information will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

In [17]:
"""
Encode the categorical features, scale all the features using the scaler used during the training
and use a threshold since the ann.predict method returns the predicted probability
"""

print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!


### Predicting the Test set results

In [18]:
y_pred = ann.predict(X_test)
# we use 1* to turn the booleans into 0s and 1s
y_pred = 1*(y_pred > 0.5)

In [19]:
print(y_pred)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


### Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_pred,y_test))
accuracy_score(y_pred,y_test)

[[1881  237]
 [ 110  272]]


0.8612